In [1]:
import pandas as pd

In [2]:
dataset3=pd.read_csv('dataset-3.csv')

In [6]:
dataset3.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


# Question1

In [5]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [7]:
import networkx as nx

In [10]:
def calculate_distance_matrix(df):
    G = nx.DiGraph()
    for index, row in df.iterrows():
        G.add_edge(row['id_start'], row['id_end'], weight=row['distance'])
        G.add_edge(row['id_end'], row['id_start'], weight=row['distance'])  
        shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G))
        nodes = sorted(G.nodes())
    distance_matrix = pd.DataFrame(index=nodes, columns=nodes)

    for node_i in nodes:
        for node_j in nodes:
            if node_i == node_j:
                distance_matrix.loc[node_i, node_j] = 0  
            else:
                distance_matrix.loc[node_i, node_j] = shortest_paths[node_i][node_j]

    return distance_matrix.astype(float)



In [11]:
result_matrix = calculate_distance_matrix(dataset3)


In [12]:
print(result_matrix)

           1001400.0  1001402.0  1001404.0  1001406.0  1001408.0  1001410.0  \
1001400.0        0.0        9.7       29.9       45.9       67.6       78.7   
1001402.0        9.7        0.0       20.2       36.2       57.9       69.0   
1001404.0       29.9       20.2        0.0       16.0       37.7       48.8   
1001406.0       45.9       36.2       16.0        0.0       21.7       32.8   
1001408.0       67.6       57.9       37.7       21.7        0.0       11.1   
1001410.0       78.7       69.0       48.8       32.8       11.1        0.0   
1001412.0       94.3       84.6       64.4       48.4       26.7       15.6   
1001414.0      112.5      102.8       82.6       66.6       44.9       33.8   
1001416.0      125.7      116.0       95.8       79.8       58.1       47.0   
1001418.0      139.3      129.6      109.4       93.4       71.7       60.6   
1001420.0      152.2      142.5      122.3      106.3       84.6       73.5   
1001422.0      161.8      152.1      131.9      115.

# Question2

In [13]:
def unroll_distance_matrix(distance_matrix):
    unique_ids = distance_matrix.index.tolist()
    combinations = [(id_start, id_end) for id_start in unique_ids for id_end in unique_ids if id_start != id_end]
    result_data = []
    for id_start, id_end in combinations:
        distance = distance_matrix.loc[id_start, id_end]
        result_data.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})
    result_df = pd.DataFrame(result_data)
    return result_df

     

In [14]:
unrolled_distances = unroll_distance_matrix(result_matrix)

In [15]:
unrolled_distances

,id_start,id_end,distance
0,1001400.0,1001402.0,9.7
1,1001400.0,1001404.0,29.9
2,1001400.0,1001406.0,45.9
3,1001400.0,1001408.0,67.6
4,1001400.0,1001410.0,78.7
...,...,...,...
1801,1004356.0,1001470.0,159.8
1802,1004356.0,1001472.0,175.8
1803,1004356.0,1001488.0,4.0
1804,1004356.0,1004354.0,2.0


# Question3

In [33]:
def find_ids_within_ten_percentage_threshold(df, reference_value):
    reference_avg_distance = df[df['id_start'] == reference_value]['distance'].mean()
    if pd.isnull(reference_avg_distance):
        return []
    threshold = reference_avg_distance * 0.1
    lower_threshold = reference_avg_distance - threshold
    upper_threshold = reference_avg_distance + threshold
    filtered_ids = df[(df['id_start'] != reference_value) & 
                      (df['distance'].between(lower_threshold, upper_threshold))]['id_start'].unique()
    sorted_filtered_ids = sorted(filtered_ids)
    return sorted_filtered_ids


    


In [34]:
reference_value = 100 
result = find_ids_within_ten_percentage_threshold(unrolled_distances, reference_value)


In [35]:
print(result)

[]


# Question4

In [36]:
def calculate_toll_rate(df):
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }
    for vehicle, rate in rate_coefficients.items():
        df[vehicle] = df['distance'] * rate

    return df


In [37]:
result_with_toll_rates = calculate_toll_rate(unrolled_distances)

In [38]:
result_with_toll_rates

,id_start,id_end,distance,moto,car,rv,bus,truck
0,1001400.0,1001402.0,9.7,7.76,11.64,14.55,21.34,34.92
1,1001400.0,1001404.0,29.9,23.92,35.88,44.85,65.78,107.64
2,1001400.0,1001406.0,45.9,36.72,55.08,68.85,100.98,165.24
3,1001400.0,1001408.0,67.6,54.08,81.12,101.40,148.72,243.36
4,1001400.0,1001410.0,78.7,62.96,94.44,118.05,173.14,283.32
...,...,...,...,...,...,...,...,...
1801,1004356.0,1001470.0,159.8,127.84,191.76,239.70,351.56,575.28
1802,1004356.0,1001472.0,175.8,140.64,210.96,263.70,386.76,632.88
1803,1004356.0,1001488.0,4.0,3.20,4.80,6.00,8.80,14.40
1804,1004356.0,1004354.0,2.0,1.60,2.40,3.00,4.40,7.20


# Question5

In [39]:
from datetime import datetime, timedelta

In [40]:
def calculate_time_based_toll_rates(df):
    df_copy = df.copy()
    start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')  
    end_date = start_date + timedelta(days=6)
    time_ranges = [
        {'start_time': datetime.strptime('00:00:00', '%H:%M:%S').time(), 'end_time': datetime.strptime('10:00:00', '%H:%M:%S').time(), 'weekday_rate': 0.8},
        {'start_time': datetime.strptime('10:00:00', '%H:%M:%S').time(), 'end_time': datetime.strptime('18:00:00', '%H:%M:%S').time(), 'weekday_rate': 1.2},
        {'start_time': datetime.strptime('18:00:00', '%H:%M:%S').time(), 'end_time': datetime.strptime('23:59:59', '%H:%M:%S').time(), 'weekday_rate': 0.8}
    ]
    start_days, start_times, end_days, end_times = [], [], [], []
    vehicle_columns = ['moto', 'car', 'rv', 'bus', 'truck']
    for day in range((end_date - start_date).days + 1):
        current_date = start_date + timedelta(days=day)
        current_weekday = current_date.strftime('%A')
        for time_range in time_ranges:
            start_time, end_time, rate = time_range['start_time'], time_range['end_time'], time_range['weekday_rate']
            if current_weekday in ['Saturday', 'Sunday']:
                rate = 0.7
            for vehicle in vehicle_columns:
                df_copy[f'{vehicle}_{current_weekday}_{start_time}_{end_time}'] = df_copy[vehicle] * rate
                
                
            start_days.append(current_weekday.capitalize())
            start_times.append(start_time)
            end_days.append(current_weekday.capitalize())
            end_times.append(end_time)
            
    df_copy['start_day'] = start_days
    df_copy['start_time'] = start_times
    df_copy['end_day'] = end_days
    df_copy['end_time'] = end_times


    return df_copy



In [ ]:
result_with_time_based_toll_rates = calculate_time_based_toll_rates(result_with_toll_rates)